# Chapter 18: Graphs

In [8]:
from collections import deque, defaultdict, namedtuple
from copy import deepcopy

# Test Graphs
test_graph_1 = {'A': set(['F', 'B', 'C']),
              'B': set(['A', 'D', 'E', 'G']),
              'C': set(['A', 'F']),
              'D': set(['B', "G"]),
              'E': set(['B', 'F']),
              'F': set(['C', 'E']),
              'G': set()}

test_graph_2 = deepcopy(test_graph_1)
test_graph_2["Z"] = "Y"

## Breadth First Search

### Vanilla BFS

In [67]:
def bfs(graph, root, directed=False):
    """
    Performs breadth first search on the given graph.
    """
    processed, discovered = set(), set(root)
    parent = defaultdict(None)
    q = deque(root)
    
    while q:
        u = q.popleft()
        process_vertex_early(u)
        for v in graph[u]:
            if v not in processed or directed:
                process_edge(u, v)
            if v not in discovered:
                q.append(v)
                discovered.add(v)
                parent[v] = u
        processed.add(u)
        process_vertex_late(u)
    print("Parents: {}".format(parent))
        
def process_vertex_early(v):
    """
    Performs computation on a vertex just before exploring its adjacency list
    """
    print("Processed vertex: {}".format(v))
    
    
def process_edge(u, v):
    """
    Performs computation on an edge the first time it is discovered
    """
    print("Processed edge: {}{}".format(u, v))
    
    
def process_vertex_late(v):
    """
    Performs computation on a vertex after exploring its adjacency list
    """
    pass


# Tests
print(test_graph_1)
bfs(test_graph_1, "A")

{'A': {'B', 'C', 'F'}, 'B': {'E', 'D', 'G', 'A'}, 'C': {'A', 'F'}, 'D': {'B', 'G'}, 'E': {'B', 'F'}, 'F': {'E', 'C'}, 'G': set()}
Processed vertex: A
Processed edge: AB
Processed edge: AC
Processed edge: AF
Processed vertex: B
Processed edge: BE
Processed edge: BD
Processed edge: BG
Processed vertex: C
Processed edge: CF
Processed vertex: F
Processed edge: FE
Processed vertex: E
Processed vertex: D
Processed edge: DG
Processed vertex: G
Parents: defaultdict(None, {'B': 'A', 'C': 'A', 'F': 'A', 'E': 'B', 'D': 'B', 'G': 'B'})


### Applications of BFS

Connected Components

In [51]:
def connected_components(graph):
    """
    Returns the number of connected components in the given graph
    """
    discovered = set()
    count = 0
    
    def bfs(root):
        discovered.add(root)
        q = deque(root)
        while q:
            u = q.popleft()
            for v in graph.get(u, set()):
                if v not in discovered:
                    discovered.add(v)
                    q.append(v)
    
    for vertex in graph:
        if vertex not in discovered:
            count += 1
            bfs(vertex)
    
    return count
            
# Tests
assert connected_components(test_graph_1) == 1
assert connected_components(test_graph_2) == 2

Two-coloring graphs

In [32]:
def bipartite(graph, color1, color2):
    """
    Return True iff the given graph is bipartite
    """
    discovered, processed = set(), set()
    color = defaultdict(None)
    bipartite = True
    
    def complement(color):
        if color == color1: return color2
        elif color == color2: return color1

    def process_edge(u, v):
        if color[u] == color[v]:
            nonlocal bipartite
            bipartite = False
        color[v] = complement(color(u))
        
    def bfs(root):
        discovered.add(root)
        q = deque(root)
        while q:
            u = q.popleft()
            processed.add(u)
            for v in graph.get(u, set()):
                if v not in processed:
                    process_edge(u, v)
                if v not in discovered:
                    discovered.add(v)
                    q.append(v)
    
    for vertex in graph:
        if vertex not in discovered:
            color[vertex] = color1
            bfs(vertex)
    
    return bipartite

# Tests
bipartite(test_graph, "WHITE", "BLACK")

## Depth First Search

### Vanilla DFS

In [ ]:
def dfs(graph, root, directed = False):
    """
    Performs depth first search on the given graph.
    """
    discovered, processed = set(), set()
    
    def recur(u):
        discovered.add(u)
        process_vertex_early(u)
        
        for v in graph[u]:
            if v not in discovered:  # Tree edge
                process_edge(u, v)
                recur(v)
            elif v not in processed or directed:  # Back edge
                process_edge(u, v)

        process_vertex_late(u)
        processed.add(u)
    
    recur(root)
        
                  
def process_vertex_early(v):
    """
    Performs computation on a vertex just before exploring its adjacency list
    """
    print("Processed vertex: {}".format(v))

    
def process_edge(u, v):
    """
    Performs computation on an edge the first time it is discovered
    """
    print("Processed edge: {}{}".format(u, v))
    

def process_vertex_late(v):
    """
    Performs computation on a vertex after exploring its adjacency list
    """
    pass

# Tests
print(test_graph_1)
dfs(test_graph_1, "A")

## 18.1 Search a maze

In [11]:
WHITE, BLACK = range(2)

Coordinate = namedtuple("Coordinate", ("x", "y"))

def search_maze(maze, start, end):
    """
    Returns a possible path from `start` to `end`
    """
    path = []
    
    def invalid_vertex(v):
        """
        Returns True if the given vertex is out of the matrix or if it is BLACK
        """
        return not((0 <= v.x < len(maze)) and 
                  (0 <= v.y < len(maze)) and
                  maze[v.x][v.y] == WHITE) 
    
    def get_neighbors(v):
        """
        Returns an iterator of this vertex's neighbors
        """
        for n in (Coordinate(v.x+1, v.y), Coordinate(v.x-1, v.y), Coordinate(v.x, v.y+1), Coordinate(v.x, v.y-1)):
            yield n
        
    def dfs(u):
        """
        Perform DFS
        """
        if invalid_vertex(u):
            return False
        
        # Indicate that vertex is discovered
        maze[u.x][u.y] = BLACK
        
        # Process vertex early
        path.append(u)
        if(u == end):
            return True
        
        # Process neighbours
        for v in get_neighbors(v):
            if bfs(v): return True
        
        # Process vertex late
        path.pop()
        return False
    
    if dfs(start): return path
    
    return []

## 18.2 Paint a Boolean Matrix